# Read embeddings

In [6]:
import numpy as np

# 0 - 334 FIFA Equipment Regulations_2025_EN
# 334 - 800 Laws of the Game 25/26
# 800 - 1034 FIFA Disciplinary Code
# 1034 - 1318 FWC26_Competition Regulations_EN
fifa_equipment_regulations = np.load("embeddings/embedding_FIFA Equipment Regulations_2025_EN.npy")
laws_of_the_game = np.load("embeddings/embedding_Laws of the Game 25-26.npy")
fifa_disciplinary_code = np.load("embeddings/embedding_FIFA Disciplinary Code 25-26.npy")
fwc26_competition_regulations = np.load("embeddings/embedding_FWC26_Competition Regulations_EN.npy")

# Read chunks

In [7]:
import json

with open('chunks.json', 'r') as fout:
    chunks = json.load(fout)

In [9]:
embeddings = [
    fifa_equipment_regulations,
    laws_of_the_game,
    fifa_disciplinary_code,
    fwc26_competition_regulations
]

for chunk, emb in zip(chunks, embeddings):
    chunk["embedding"] = emb.tolist()

In [11]:
chunks[0].keys()

dict_keys(['chunk', 'metadata', 'embedding'])

# Milvus
https://milvus.io/docs/full_text_search_with_milvus.md

In [12]:
from pymilvus import (
    MilvusClient,
    DataType,
    Function,
    FunctionType,
    AnnSearchRequest,
    RRFRanker,
)

In [ ]:
uri = "http://localhost:19530"
collection_name = "football_docs"
client = MilvusClient(uri=uri)

ConnectionConfigException: <ConnectionConfigException: (code=1, message=uri: localhost:19530 is illegal, needs start with [unix, http, https, tcp] or a local file endswith [.db])>

In [ ]:
analyzer_params = {"tokenizer": "standard", "filter": ["lowercase"]}

In [ ]:
schema = MilvusClient.create_schema()
schema.add_field(
    field_name="id",
    datatype=DataType.VARCHAR,
    is_primary=True,
    auto_id=True,
    max_length=100,
)
schema.add_field(
    field_name="content",
    datatype=DataType.VARCHAR,
    max_length=65535,
    analyzer_params=analyzer_params,
    enable_match=True,  # Enable text matching
    enable_analyzer=True,  # Enable text analysis
)
schema.add_field(field_name="sparse_vector", datatype=DataType.SPARSE_FLOAT_VECTOR)
schema.add_field(
    field_name="dense_vector",
    datatype=DataType.FLOAT_VECTOR,
    dim=1024,  # Dimension for Qwen3-Embedding-0.6B
)
schema.add_field(field_name="metadata", datatype=DataType.JSON)

bm25_function = Function(
    name="bm25",
    function_type=FunctionType.BM25,
    input_field_names=["content"],
    output_field_names="sparse_vector",
)

schema.add_function(bm25_function)


In [ ]:
index_params = MilvusClient.prepare_index_params()
index_params.add_index(
    field_name="sparse_vector",
    index_type="SPARSE_INVERTED_INDEX",
    metric_type="BM25",
)
index_params.add_index(field_name="dense_vector", index_type="FLAT", metric_type="IP")

if client.has_collection(collection_name):
    client.drop_collection(collection_name)
client.create_collection(
    collection_name=collection_name,
    schema=schema,
    index_params=index_params,
)
print(f"Collection '{collection_name}' created successfully")


In [ ]:
entities = []
for i, c in enumerate(chunks):
    entities.append(
        {
            "content": c["content"],
            "dense_vector": c["embedding"],
            "metadata": c.get("metadata", {}),
        }
    )

# Insert data
client.insert(collection_name, entities)
print(f"Inserted {len(entities)} documents")


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")

In [ ]:
query = "what is hybrid search"

query_embedding = embedding = model.encode(query, show_progress_bar=True)

sparse_search_params = {"metric_type": "BM25"}
sparse_request = AnnSearchRequest(
    [query], "sparse_vector", sparse_search_params, limit=5
)

dense_search_params = {"metric_type": "IP"}
dense_request = AnnSearchRequest(
    [query_embedding], "dense_vector", dense_search_params, limit=5
)

results = client.hybrid_search(
    collection_name,
    [sparse_request, dense_request],
    ranker=RRFRanker(),  # Reciprocal Rank Fusion for combining results
    limit=5,
    output_fields=["content", "metadata"],
)
hybrid_results = results[0]

print("\nHybrid Search (Combined):")
for i, result in enumerate(hybrid_results):
    print(
        f"{i+1}. Score: {result['distance']:.4f}, Content: {result['entity']['content']}"
    )


# Answer generation

In [ ]:
context = "\n\n".join([doc["entity"]["content"] for doc in hybrid_results])

prompt = f"""Answer the following question based on the provided context. 
If the context doesn't contain relevant information, just say "I don't have enough information to answer this question."

Context:
{context}

Question: {query}

Answer:"""

response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that answers questions based on the provided context.",
        },
        {"role": "user", "content": prompt},
    ],
)

print(response.choices[0].message.content)
